# Welcome to the notebook for Hong Kong JobsDb job advertisement analysis


## import module and data

In [2]:
# scrap the data

# run hkjobsdb_scrap.py outside of this notebook
# returns <job title-date of scrap-number>.csv

In [7]:
import pandas as pd

In [8]:
# load the dat

df_raw = pd.read_csv('data-scientist-2022-12-06-1.csv')


## inspect the data

In [9]:
df_raw.head(10)

,title,salary,company,posted,district,job_description,Career Level,Years of Experience,Company Website,Qualification,Job Type,Job Functions,url
0,Data Scientist,salary,Connexe Search Limited,2022-12-06,Kowloon Bay,Our client is one of the leading financial ins...,NaN,NaN,NaN,NaN,Contract,"Banking / Finance, Others, Information Technol...",https://hk.jobsdb.com/hk/en/job/data-scientist...
1,Data Analyst (Data & AI),salary,NCSI (HK) Limited,2022-12-06,Tai Koo,Key Role Responsibilities:\n\nParticipate in c...,NaN,NaN,http://www.ncs-i.com,NaN,"Full Time, Permanent","Information Technology (IT), Product Managemen...",https://hk.jobsdb.com/hk/en/job/data-analyst-d...
2,Data Analyst (Data Governance),salary,Gientech Technology (Hong Kong) Limited,2022-12-06,Ngau Tau Kok,Duties:\nManage & support data governance mode...,Middle,3 years,NaN,Degree,"Full Time, Permanent","Information Technology (IT), Data Scientist",https://hk.jobsdb.com/hk/en/job/data-analyst-d...
3,Solution Analyst - ICC - Data Integration,salary,Cathay Pacific Airways Ltd,2022-12-06,Lantau Island,Reports to: Solution Lead / Senior Solution Le...,NaN,NaN,http://www.cathaypacific.com/careers,NaN,"Full Time, Permanent","Transportation & Logistics, Aviation Services,...",https://hk.jobsdb.com/hk/en/job/solution-analy...
4,Analyst Programmer (Data Analytics),salary,China State Construction Engineering (Hong Kon...,2022-12-06,district,Job Description:\nEstablish and maintain high-...,Entry Level,3 years,https://www.cscechk.com/en/,Degree,"Full Time, Permanent","Information Technology (IT), DBA, Others, Data...",https://hk.jobsdb.com/hk/en/job/analyst-progra...
5,Business Intelligence Analyst,salary,Bureau Veritas Hong Kong Limited,2022-12-06,Kowloon Bay,Responsibilities:\nBuild dashboards to suppor...,Middle,5 years,NaN,Degree,"Full Time, Permanent","Professional Services, Business Analysis / Dat...",https://hk.jobsdb.com/hk/en/job/business-intel...
6,Data Integration Engineer (Data & AI),salary,NCSI (HK) Limited,2022-12-06,Tai Koo,Key Role Responsibilities:\n\nDrive Data & AI ...,NaN,NaN,http://www.ncs-i.com,NaN,"Full Time, Permanent","Information Technology (IT), Product Managemen...",https://hk.jobsdb.com/hk/en/job/data-integrati...
7,Data Scientist [Python & Cloud project],salary,Peoplehub Limited,2022-12-06,district,Responsibilities:\nEstablish and maintain high...,NaN,NaN,https://www.peoplehub.hk/,NaN,"Full Time, Permanent","Information Technology (IT), Application Speci...",https://hk.jobsdb.com/hk/en/job/data-scientist...
8,Senior Associate (Data Analyst) – Risk Advisor...,salary,BDO,2022-12-06,Sheung Wan,"Responsibilities:\nCollect data, build and mai...",Middle,2 years,http://www.bdo.com.hk,Degree,"Full Time, Permanent","Professional Services, Business Analysis / Dat...",https://hk.jobsdb.com/hk/en/job/senior-associa...
9,Data Engineer,salary,Orient Overseas Container Line Ltd (OOCL),2022-12-06,Shatin Area,"Job Description\nResponsible for design, devel...",Middle,2 years,http://www.oocl.com,Degree,"Full Time, Permanent","Information Technology (IT), Application Speci...",https://hk.jobsdb.com/hk/en/job/data-engineer-...


### need recategorize to fewer catergories
* data job type
* career level

### feature extraction
* job type to binary: full time (T/F)
* job functions: label category for each job functions

### convert to numerical variable
* years of experience

In [10]:
df_inspect = df_raw.copy()

In [13]:
is_data_analyst = 'data analyst' in 
print(df_inspect['title'].where('data analyst' in df_inspect['title']))

ValueError: Array conditional must be same shape as self